In [14]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [15]:
empresas = ['AMER3_2', 'MGLU3_2', 'PETR4_2']

In [16]:
def getCsv(empresa):
    dados = pd.read_csv(empresa + '.csv', index_col='Unnamed: 0')
    if 'data_hora' not in dados.columns:
        dados['data_hora'] = f"{dados['data']} {dados['hora']}"
    dados['data_hora'] = pd.to_datetime(dados['data_hora'], format='%Y-%m-%d %H:%M:%S')

    return dados.sort_values('data_hora')

###################################################################################

In [17]:
def getDadosBolsa(empresa, dataMax, dataMin):

    dados = yf.Ticker(empresa + '.SA')
    dados = dados.history(start=(dataMin - timedelta(days=2)), end=(dataMax + timedelta(days=7)), interval='1d')
    dados.reset_index(inplace=True)
    dados['data_hora'] = dados['Date']
    dados.drop(columns=['Date'], inplace=True)
    dados.set_index('data_hora', inplace=True)
    return dados

In [18]:
def getDataMaisProxima(empresa, data):

    datas = {
                abs(data.timestamp() - date.timestamp()) : date for date in empresa.index}
    
    dataMaisProxima = datas[min(datas.keys())]

    return dataMaisProxima - timedelta(days=1), dataMaisProxima, dataMaisProxima + timedelta(days=1), dataMaisProxima + timedelta(days=2)

In [19]:
def getValores(dados, data):
    try:
        return dados.iloc[dados.index.get_loc(data)]

    except KeyError:
        return getValores(dados, data + timedelta(days=1))

def montaDictValores(valores):
    return {
        'Abertura': valores.Open,
        'Fechamento': valores.Close,
    }

def getValoresData(dados, data):
    return montaDictValores(getValores(dados, data))

In [20]:
def extraiValoresBolsa(empresa):
    df = getCsv(empresa)
    
    dados_bolsa = getDadosBolsa(empresa, df['data_hora'].max(), df['data_hora'].min())

    aberturaDiaAnterior = []
    aberturaDiaAtual = []
    aberturaDiaSeguinte = []
    abertura2Dias = []

    fechamentoDiaAnterior = []
    fechamentoDiaAtual = []
    fechamentoDiaSeguinte = []
    fechamento2Dias = []

    for data in df['data_hora']:
        diaAnterior, dataAtual, dataSeguinte, dia2 = getDataMaisProxima(dados_bolsa, data)

        valores = getValoresData(dados_bolsa, diaAnterior)
        aberturaDiaAnterior.append(valores['Abertura'])
        fechamentoDiaAnterior.append(valores['Fechamento'])

        valores = getValoresData(dados_bolsa, dataAtual)
        aberturaDiaAtual.append(valores['Abertura'])
        fechamentoDiaAtual.append(valores['Fechamento'])

        valores = getValoresData(dados_bolsa, dataSeguinte)
        aberturaDiaSeguinte.append(valores['Abertura'])
        fechamentoDiaSeguinte.append(valores['Fechamento'])
        
        
        valores = getValoresData(dados_bolsa, dia2)
        abertura2Dias.append(valores['Abertura'])
        fechamento2Dias.append(valores['Fechamento'])
        
    df['aberturaDiaAnterior'] = aberturaDiaAnterior
    df['fechamentoDiaAnterior'] = fechamentoDiaAnterior

    df['aberturaDiaAtual'] = aberturaDiaAtual
    df['fechamentoDiaAtual'] = fechamentoDiaAtual

    df['aberturaDiaSeguinte'] = aberturaDiaSeguinte
    df['fechamentoDiaSeguinte'] = fechamentoDiaSeguinte

    df['abertura2Dias'] = abertura2Dias
    df['fechamento2Dias'] = fechamento2Dias

    return df

In [21]:
for empresa in empresas:
    print(empresa)
    df = extraiValoresBolsa(empresa)
    df.to_csv(empresa + '.csv')

AMER3_2


ValueError: time data "0     04.10.2023
1     01.10.2023
2     27.09.2023
3     22.09.2023
4     19.09.2023
5     14.09.2023
6     11.09.2023
7     07.09.2023
8     04.09.2023
9     31.08.2023
10    28.08.2023
11    25.08.2023
12    22.08.2023
13    18.08.2023
14    15.08.2023
15    10.08.2023
16    05.08.2023
17    02.08.2023
18    27.07.2023
19    24.07.2023
20    21.07.2023
21    18.07.2023
22    14.07.2023
23    11.07.2023
24    07.07.2023
Name: data, dtype: object 0     07:55
1     16:29
2     09:17
3     09:16
4     17:40
5     09:21
6     22:12
7     14:20
8     21:02
9     10:28
10    08:28
11    10:33
12    20:26
13    05:38
14    16:33
15    17:12
16    05:41
17    15:39
18    16:03
19    07:43
20    16:23
21    08:40
22    11:29
23    21:05
24    13:44
Name: hora, dtype: object" doesn't match format "%Y-%m-%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are a